In [1]:
DATA_FOLDER = 'data/so/'

In [2]:
from IPython.display import display, HTML
from viz import visualize_seqs
import numpy as np

Read data

In [3]:
file_index = 1
events = {}
times = {}

for data_type in ['train', 'test']:
    with open(DATA_FOLDER + 'event-' + str(file_index) + '-' + data_type + '.txt') as f_events:
        events[data_type] = [[int(y) for y in x.split()] for x in f_events]
        
    with open(DATA_FOLDER + 'time-' + str(file_index) + '-' + data_type + '.txt') as f_times:
        times[data_type] = [[float(y) for y in x.split()] for x in f_times]

badge_map = {}
with open(DATA_FOLDER + 'badges.csv') as f_badge_labels:
    next(f_badge_labels)
    for row in f_badge_labels:
        id, name = row.split(',')
        badge_map[int(id)] = name.strip()

# Models without Time

Errors for these models are represented as a percent where:

number wrong / total predictions = error%

## Get Event Stats

In [4]:
event_dict = {}
for data_type in ['train', 'test']:
    for seq in events[data_type]:
        for event in seq:
            if event not in event_dict:
                event_dict[event] = 0
            event_dict[event] += 1
            
all_events = list(event_dict.keys())
num_events = len(all_events)
max_event = max(all_events)
print('Events', all_events)
print('Num events', num_events)
print('Max event', max_event)

print('')
print('-' * 8)
print('Event counts')
print(event_dict)

Events [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
Num events 22
Max event 22

--------
Event counts
{1: 30046, 2: 9705, 3: 3207, 4: 208506, 5: 27823, 6: 29955, 7: 9442, 8: 9065, 9: 110184, 10: 6578, 11: 1031, 12: 13260, 13: 4184, 14: 9249, 15: 1500, 16: 482, 17: 1342, 18: 2597, 19: 1816, 20: 334, 21: 79, 22: 29}


# Helper Functions

In [9]:
def compute_error(y_true, y_pred):
    num_error = 0
    total = 0
    for s, seq in enumerate(y_true):
        total += len(seq) - 1
        for e, event in enumerate(seq[1:]):
            if event != y_pred[s][e]:
                num_error += 1
                
    return (num_error / total) * 100

def show_results(train_pred, test_pred):
    print('----')
    print('Train Error: ', compute_error(events['train'], train_pred))
    print('Test Error: ', compute_error(events['test'], test_pred))
    display(HTML(visualize_seqs(events['train'], train_pred)))

In [6]:
def convert_flat(predictions, data_type):
    pred = []
    index = 0
    for s, seq in enumerate(events[data_type]):
        start_index = index
        index += len(seq) - 1
        pred.append(predictions[start_index:index])
        
    return pred

In [7]:
def convert_nn_seq(predictions, data_type):
    pred = []
    for s, seq in enumerate(events[data_type]):
        seq_len = len(seq) - 1
        pred.append(predictions[s][-seq_len:].tolist())
        
    return pred

# Baselines

## Majority Predictor

Source: https://github.com/dunan/NeuralPointProcess/blob/master/code/baselines/majority_predictor/event_majority_baseline.py

In [56]:
def draw_button(e_true, e_pred, t_true, t_pred):
    styles = {
        'class': 'btn-primary',
        'opacity': 1.0
    }
    e_display = str(e_true)
    t_display = ''
    show_both = e_true is not None and e_pred is not None and t_true is not None and t_pred is not None
    
    if e_true != e_pred:
        styles['opacity'] = 0.3

    return '<button type="button" ' + \
           'class="btn ' + str(styles['class']) + '" ' + \
           'style="opacity: ' + str(styles['opacity']) + ';"' + \
           '>' + \
           str(e_display) + \
           ('<br>' if show_both else '') + \
           str(t_display) + \
           '</button>'


def visualize_seqs(events_true=None, events_pred=None, times_true=None, times_pred=None, num=5):
    # One set have to not be None
    assert (events_true is not None and events_pred is not None) or (times_true is not None and times_pred is not None)

    # Determine which array to iterate over
    array_iter = events_pred if events_pred is not None else times_pred

    seqs = ''
    for s in range(len(array_iter[:num])):
        seq_viz = '<strong><u>' + str(s + 1) + '</u></strong> '
        for e in range(len(array_iter[s])):
            seq_viz += draw_button(None if events_true is None else events_true[s][e],
                                   None if events_pred is None else events_pred[s][e],
                                   None if times_true is None else times_true[s][e],
                                   None if times_pred is None else times_pred[s][e])

        seqs += '<div>' + seq_viz + '</div><br>'

    return seqs


In [57]:
from collections import Counter

cnt = Counter()
for seq in events['train']:
    for event in seq:
        cnt[event] += 1

top = cnt.most_common(1)[0][0]
print('Most commom event is', top)

train_pred = [[top for event in seq[1:]] for seq in events['train']]
test_pred = [[top for event in seq[1:]] for seq in events['test']]
show_results(train_pred, test_pred)

Most commom event is 4
----
Train Error:  56.53630575271122
Test Error:  57.106363456264916


## Markov Chain

In [47]:
from collections import Counter, defaultdict

def build_mc_model(order):
    assert order >= 1
    model = defaultdict(Counter)
    label_cnt = Counter()
    for seq in events['train']:
        if len(seq) <= order or len(seq) <= 2:
            continue
            
        for event in seq:
            label_cnt[event] += 1
            
        state = tuple(seq[:order])
        
        for i in range(order, len(seq)):
            model[state][seq[i]] += 1
            state = state[1:] + (seq[i],)
                
    for key in model:
        t = model[key].most_common(1)[0][0]
        model[key] = t

    return model, label_cnt

def predict_mc_model(order):
    model, label_cnt = build_mc_model(order)
    top_pred = label_cnt.most_common(1)[0][0]
    total = 0
    err_cnt = 0
    
    preds = {}
    for data_type in ['train', 'test']:
        preds[data_type] = []
        for seq in events[data_type]:
            seq_pred = []
            for i in range(1, order):
                if i >= len(seq):
                    break
                    
                seq_pred.append(top_pred)

            state = tuple(seq[:order])
            for i in range(order, len(seq)):
                if state in model:
                    pred = model[state]
                else:
                    pred = top_pred
                state = state[1:] + (seq[i],)

                seq_pred.append(pred)
            preds[data_type].append(seq_pred)
    return preds

### MC-1

In [ ]:
preds = predict_mc_model(1)
show_results(preds['train'], preds['test'])

### MC-2

In [49]:
preds = predict_mc_model(2)
show_results(preds['train'], preds['test'])

----
Train Error:  54.63673076210589
Test Error:  55.162813976039295


### MC-3

In [48]:
preds = predict_mc_model(3)
show_results(preds['train'], preds['test'])

----
Train Error:  53.681650497255696
Test Error:  55.375520035033944


### MC-4

In [ ]:
preds = predict_mc_model(4)
show_results(preds['train'], preds['test'])

## Chunking Models

In [ ]:
CHUNK_SIZE = 5

In [ ]:
x = {}
y = {}

for data_type in ['train', 'test']:
    x[data_type] = []
    y[data_type] = []
    for seq in events[data_type]:
        current_seq = [0] * (CHUNK_SIZE - 1) + seq
        for e, event in enumerate(current_seq):
            if e >= CHUNK_SIZE:
                y[data_type].append(event)
                index_lookback = e - CHUNK_SIZE
                x[data_type].append(current_seq[index_lookback:e])

    x[data_type] = np.asarray(x[data_type])
    y[data_type] = np.asarray(y[data_type])

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x['train'], y['train'])
train_pred = convert_flat(model.predict(x['train']), 'train')
test_pred = convert_flat(model.predict(x['test']), 'test')

print('Without rounding')
show_results(train_pred, test_pred)

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x['train'], y['train'])
train_pred = convert_flat(np.round(model.predict(x['train'])), 'train')
test_pred = convert_flat(np.round(model.predict(x['test'])), 'test')

print('With rounding')
show_results(train_pred, test_pred)

Going to normalize before inputting to LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression

x_train = x['train'] / max_event
y_train = y['train'] / max_event
x_test = x['test'] / max_event
y_test = y['test'] / max_event

model = LinearRegression()
model.fit(x_train, y_train)
train_pred = model.predict(x_train)
test_pred = model.predict(x_test)

train_pred = np.round(train_pred * max_event)
test_pred = np.round(test_pred * max_event)

train_pred = convert_flat(train_pred, 'train')
test_pred = convert_flat(test_pred, 'test')


print('With rounding')
show_results(train_pred, test_pred)

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(x['train'], y['train'])
train_pred = convert_flat(model.predict(x['train']), 'train')
test_pred = convert_flat(model.predict(x['test']), 'test')

show_results(train_pred, test_pred)

Visualize the decision tree

In [ ]:
from sklearn.tree import export_graphviz
from IPython.display import Image
import pydot

# model_dot = export_graphviz(model,
#                             out_file=None,
#                             filled=True,
#                             rounded=True)
# 
# graph = pydot.graph_from_dot_data(model_dot)
# display(Image(graph[0].create_png()))

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x['train'], y['train'])
train_pred = convert_flat(model.predict(x['train']), 'train')
test_pred = convert_flat(model.predict(x['test']), 'test')

show_results(train_pred, test_pred)

## AdaBoost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier()
model.fit(x['train'], y['train'])
train_pred = convert_flat(model.predict(x['train']), 'train')
test_pred = convert_flat(model.predict(x['test']), 'test')

show_results(train_pred, test_pred)

# One-Hot Data

Convert the data into one-hot so that rather than using an integer for the category, use a vector:

2 ===>  <0, 0, 1, 0, 0>

In [ ]:
x = {}
y = {}

for data_type in ['train', 'test']:
    x[data_type] = []
    y[data_type] = []
    for seq in events[data_type]:
        current_seq = [-1] * (CHUNK_SIZE - 1) + seq
        for e, event in enumerate(current_seq):
            if e >= CHUNK_SIZE:
                y[data_type].append(event)
                index_lookback = e - CHUNK_SIZE
                
                current_x = []
                for back_event in current_seq[index_lookback:e]:
                    vector = [0] * max_event
                    if back_event > 0:
                        vector[back_event-1] = 1
                    current_x += vector
                x[data_type].append(current_x)

    x[data_type] = np.asarray(x[data_type])
    y[data_type] = np.asarray(y[data_type])

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(x['train'], y['train'])
train_pred = convert_flat(model.predict(x['train']), 'train')
test_pred = convert_flat(model.predict(x['test']), 'test')

show_results(train_pred, test_pred)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x['train'], y['train'])
train_pred = convert_flat(model.predict(x['train']), 'train')
test_pred = convert_flat(model.predict(x['test']), 'test')

show_results(train_pred, test_pred)

# Neural Networks

We are going to try a bunch of different network configurations so let's import stuff that we'll need.

In [ ]:
from keras.models import Sequential
from keras import optimizers
from keras import layers

## Regular FeedForward Neural Network

We need to alter the data for the FNN.

In [ ]:
y_train = np.zeros((y['train'].shape[0], num_events))
y_train[np.arange(y['train'].shape[0]), y['train'] - 1] = 1

y_test = np.zeros((y['test'].shape[0], num_events))
y_test[np.arange(y['test'].shape[0]), y['test'] - 1] = 1

#### Single - tanh

A single hidden layer using tanh activation

In [ ]:
model = Sequential()
model.add(layers.Dense(50, activation='tanh', batch_input_shape=(None, x['train'].shape[1])))
model.add(layers.Dense(num_events, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x['train'], y_train, validation_data=(x['test'], y_test), batch_size=100, epochs=3)

train_pred = convert_flat(np.argmax(model.predict(x['train']), axis=1) + 1, 'train')
test_pred = convert_flat(np.argmax(model.predict(x['test']), axis=1) + 1, 'test')
show_results(train_pred, test_pred)

#### Single - linear

A single hidden layer using linear activation

In [ ]:
model = Sequential()
model.add(layers.Dense(50, activation='linear', batch_input_shape=(None, x['train'].shape[1])))
model.add(layers.Dense(num_events, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x['train'], y_train, validation_data=(x['test'], y_test), batch_size=100, epochs=3)

train_pred = convert_flat(np.argmax(model.predict(x['train']), axis=1) + 1, 'train')
test_pred = convert_flat(np.argmax(model.predict(x['test']), axis=1) + 1, 'test')
show_results(train_pred, test_pred)

#### Single - relu

A single hidden layer using relu activation

In [ ]:
model = Sequential()
model.add(layers.Dense(50, activation='relu', batch_input_shape=(None, x['train'].shape[1])))
model.add(layers.Dense(num_events, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x['train'], y_train, validation_data=(x['test'], y_test), batch_size=100, epochs=3)

train_pred = convert_flat(np.argmax(model.predict(x['train']), axis=1) + 1, 'train')
test_pred = convert_flat(np.argmax(model.predict(x['test']), axis=1) + 1, 'test')
show_results(train_pred, test_pred)

#### Multiple - tanh

Multiple hidden layers with tanh activation

In [ ]:
model = Sequential()
model.add(layers.Dense(50, activation='tanh', batch_input_shape=(None, x['train'].shape[1])))
model.add(layers.Dense(15, activation='tanh'))
model.add(layers.Dense(num_events, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x['train'], y_train, validation_data=(x['test'], y_test), batch_size=100, epochs=3)

train_pred = convert_flat(np.argmax(model.predict(x['train']), axis=1) + 1, 'train')
test_pred = convert_flat(np.argmax(model.predict(x['test']), axis=1) + 1, 'test')
show_results(train_pred, test_pred)

#### Multiple - linear

Multiple hidden layers with linear activation

In [ ]:
model = Sequential()
model.add(layers.Dense(50, activation='linear', batch_input_shape=(None, x['train'].shape[1])))
model.add(layers.Dense(15, activation='linear'))
model.add(layers.Dense(num_events, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x['train'], y_train, validation_data=(x['test'], y_test), batch_size=100, epochs=3)

train_pred = convert_flat(np.argmax(model.predict(x['train']), axis=1) + 1, 'train')
test_pred = convert_flat(np.argmax(model.predict(x['test']), axis=1) + 1, 'test')
show_results(train_pred, test_pred)

#### Multiple - relu

Multiple hidden layers with relu activation

In [ ]:
model = Sequential()
model.add(layers.Dense(50, activation='relu', batch_input_shape=(None, x['train'].shape[1])))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dense(num_events, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x['train'], y_train, validation_data=(x['test'], y_test), batch_size=100, epochs=3)

train_pred = convert_flat(np.argmax(model.predict(x['train']), axis=1) + 1, 'train')
test_pred = convert_flat(np.argmax(model.predict(x['test']), axis=1) + 1, 'test')
show_results(train_pred, test_pred)

# Entire Sequences

Instead of using a lookback of CHUNK_SIZE, going to input the entire user history.  (Will use CNN and RNN for this.)

Padding the sequences with 0's at the beginning

Sequence Data

In [ ]:
x = {}
y = {}

for data_type in ['train', 'test']:
    x[data_type] = []
    y[data_type] = []
    for seq in events[data_type]:
        x[data_type].append(seq[:-1])
        y[data_type].append(seq[1:])

    x[data_type] = np.asarray(x[data_type])
    y[data_type] = np.asarray(y[data_type])

In [ ]:
# Find the max user length
user_lengths = [len(seq) for seq in events['train']] + [len(seq) for seq in events['test']]
user_length_counts = {}
for length in user_lengths:
    if length not in user_length_counts:
        user_length_counts[length] = 0
        
    user_length_counts[length] += 1
user_lengths = list(set(user_lengths))
user_lengths = sorted(user_lengths, reverse=True)
max_user_length = max(user_lengths)

print('Max Seq Length', max_user_length)
# print(user_lengths)

Subtract one from x

In [ ]:
for data_type in ['train', 'test']:
    x[data_type] = [[event - 1 for event in seq] for seq in x[data_type]]

In [ ]:
# Pad sequences
from keras.preprocessing import sequence

for data_type in ['train', 'test']:
    x[data_type] = sequence.pad_sequences(x[data_type], maxlen=max_user_length)
    y[data_type] = sequence.pad_sequences(y[data_type], maxlen=max_user_length)

y variable needs to be one-hot for everything.  (x will be made one-hot later for different models)

In [ ]:
num_events = 22
one_hot = np.eye(num_events)
for data_type in ['train', 'test']:
    y[data_type] = [[one_hot[event-1] if event > 0 else np.zeros(num_events) for event in seq] for seq in y[data_type]]
    y[data_type] = np.asarray(y[data_type])

## Embedding Models

### Convolutional Neural Network (CNN)

Using the model found here: https://github.com/mhjabreel/CharCnn_Keras/blob/master/models/char_cnn_kim.py

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Concatenate
from keras.layers import Convolution1D
from keras.layers import GlobalMaxPooling1D
from keras.layers import Embedding
from keras.layers import AlphaDropout


dropout_p = 0.1
fully_connected_layers = [1024, 1024]
conv_layers = [
    [256, 10],
    [256, 7],
    [256, 5],
    [256, 3]
]


# Input layer
inputs = Input(shape=(max_user_length,))

# Embedding layers
cnn_x = Embedding(num_events, 512, input_length=max_user_length)(inputs)

# Convolution layers
convolution_output = []
for num_filters, filter_width in conv_layers:
    conv = Convolution1D(filters=num_filters,
                         kernel_size=filter_width,
                         activation='tanh',
                         name='Conv1D_{}_{}'.format(num_filters, filter_width))(cnn_x)
    pool = GlobalMaxPooling1D(name='MaxPoolingOverTime_{}_{}'.format(num_filters, filter_width))(conv)
    convolution_output.append(pool)
cnn_x = Concatenate()(convolution_output)

# Fully connected layers
for fl in fully_connected_layers:
    cnn_x = Dense(fl, activation='selu', kernel_initializer='lecun_normal')(cnn_x)
    cnn_x = AlphaDropout(dropout_p)(cnn_x)
    
# Output layer
predictions = Dense(num_events, activation='softmax')(cnn_x)

# Build and compile model
model = Model(inputs=inputs, outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x['train'], y['train'], validation_data=(x['test'], y['test']))

### RNN Cell

In [ ]:
model = Sequential()
model.add(layers.Embedding(num_events, 512, mask_zero=True))
model.add(layers.Masking(mask_value=0.))
model.add(layers.SimpleRNN(50, activation='tanh', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(10, activation='relu')))
model.add(layers.TimeDistributed(layers.Dense(num_events, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x['train'], y['train'], validation_data=(x['test'], y['test']), batch_size=100, epochs=3)

train_pred = convert_nn_seq(np.argmax(model.predict(x['train']), axis=2) + 1, 'train')
test_pred = convert_nn_seq(np.argmax(model.predict(x['test']), axis=2) + 1, 'test')
show_results(train_pred, test_pred)

In [ ]:
model = Sequential()
model.add(layers.Embedding(num_events, 128, mask_zero=True))
model.add(layers.Masking(mask_value=0.))
model.add(layers.SimpleRNN(50, activation='tanh', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(num_events, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x['train'], y['train'], validation_data=(x['test'], y['test']), batch_size=100, epochs=3)

train_pred = convert_nn_seq(np.argmax(model.predict(x['train']), axis=2) + 1, 'train')
test_pred = convert_nn_seq(np.argmax(model.predict(x['test']), axis=2) + 1, 'test')
show_results(train_pred, test_pred)

### LSTM Cell

In [ ]:
model = Sequential()
model.add(layers.Embedding(num_events, 128, mask_zero=True))
model.add(layers.Masking(mask_value=0.))
model.add(layers.LSTM(50, activation='tanh', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(num_events, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x['train'], y['train'], validation_data=(x['test'], y['test']), batch_size=100, epochs=3)

train_pred = convert_nn_seq(np.argmax(model.predict(x['train']), axis=2) + 1, 'train')
test_pred = convert_nn_seq(np.argmax(model.predict(x['test']), axis=2) + 1, 'test')
show_results(train_pred, test_pred)

In [ ]:
model = Sequential()
model.add(layers.Embedding(num_events, 128, mask_zero=True))
model.add(layers.Masking(mask_value=0.))
model.add(layers.LSTM(50, activation='tanh', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(10, activation='relu')))
model.add(layers.TimeDistributed(layers.Dense(num_events, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x['train'], y['train'], validation_data=(x['test'], y['test']), batch_size=100, epochs=3)

train_pred = convert_nn_seq(np.argmax(model.predict(x['train']), axis=2) + 1, 'train')
test_pred = convert_nn_seq(np.argmax(model.predict(x['test']), axis=2) + 1, 'test')
show_results(train_pred, test_pred)

## One Hot Models

Convert x integers into one-hot as well

In [ ]:
num_events = 22
one_hot = np.eye(num_events)
for data_type in ['train', 'test']:
    x[data_type] = [[one_hot[event-1] if event > 0 else np.zeros(num_events) for event in seq] for seq in x[data_type]]
    x[data_type] = np.asarray(x[data_type])

### RNN Cell

In [ ]:
model = Sequential()
model.add(layers.Masking(mask_value=0., input_shape=(max_user_length, num_events)))
model.add(layers.SimpleRNN(50, activation='tanh', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(10, activation='relu')))
model.add(layers.TimeDistributed(layers.Dense(num_events, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x['train'], y['train'], validation_data=(x['test'], y['test']), batch_size=100, epochs=10)

train_pred = convert_nn_seq(np.argmax(model.predict(x['train']), axis=2) + 1, 'train')
test_pred = convert_nn_seq(np.argmax(model.predict(x['test']), axis=2) + 1, 'test')
show_results(train_pred, test_pred)

### LSTM Cell

In [ ]:
model = Sequential()
model.add(layers.Masking(mask_value=0., input_shape=(max_user_length, num_events)))
model.add(layers.LSTM(50, activation='tanh', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(10, activation='relu')))
model.add(layers.TimeDistributed(layers.Dense(num_events, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x['train'], y['train'], validation_data=(x['test'], y['test']), batch_size=100, epochs=10)

train_pred = convert_nn_seq(np.argmax(model.predict(x['train']), axis=2) + 1, 'train')
test_pred = convert_nn_seq(np.argmax(model.predict(x['test']), axis=2) + 1, 'test')
show_results(train_pred, test_pred)